# Imports

In [1]:
import pandas as pd
pd.set_option('display.max_rows', 50)
pd.set_option('display.max_columns', None) 
import numpy as np
import re
from collections import defaultdict

# Gameplan

The following spreadsheets need to be cleaned and merged together:

    - players_yahooleague : contains all players from yahoo fantasy for every season, this is the base sheet that I'll be working off of
    - player_keys_raw : all the online ids for each fantasy api for each player
    - stats_team : team statistics to be used for defenses
    - stats_def: defensive statistics for defenses
    - conversions: defensive statistics for defenses
    - stats_player: player statistics to be used for granular player statistics
    - stats_yahooleague: Yahoo fantasy stats for every season, includes season end score
    - depth_14to24 : Depth charts from 14 to 24 season
        -depth_2025 : Depth chart for 2025 season -- columns and layour are a little different than other seasons
    -coaches: all head offensive and defensive coaches for every team for every year

**The first order of business will be finding the common IDs across all datasets in order to merge with unique keys, some are missing from some sets**



In [2]:
yahoo_players = pd.read_csv('Raw/players_yahooleague_raw_14to25.csv')
yahoo_players.head()
yahoo_players = yahoo_players.drop(columns='Unnamed: 0')
yahoo_players.head()


,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,O,TE,https://sports.yahoo.com/nfl/players/3950,2014
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,O,QB,https://sports.yahoo.com/nfl/players/4256,2014
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,K,K,https://sports.yahoo.com/nfl/players/4269,2014
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,O,QB,https://sports.yahoo.com/nfl/players/4416,2014


In [3]:
master_ids = pd.read_csv('Raw/player_keys_raw.csv')
master_ids.head()
master_ids = master_ids.drop(columns='Unnamed: 0')
master_ids.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025


In [4]:
master_ids[master_ids['yahoo_id'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,NaN,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,NaN,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,NaN,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,NaN,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,NaN,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12143,990,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Bubby Brister,bubby brister,QB,KCC,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025


Missing ids look to be rookies this year or players who played a very long time ago. Also noticed there are a handful of fantasy positions on list that arent in league like defensive individual players

In [5]:
master_ids = master_ids[master_ids['position'].isin(['QB', 'RB', 'WR', 'TE', 'PK'])]
master_ids['position'].value_counts()

position
WR    2038
RB    1587
TE     948
QB     666
PK     276
Name: count, dtype: int64

In [6]:
master_ids['yahoo_id'].isna().sum()

# 4000 missing IDs were removed this way, good start

np.int64(2833)

In [7]:
yahoo_players[yahoo_players['name.full'] == 'Ollie Gordon II']

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
13835,12,RB,nfl.p.41969,Mia,Miami Dolphins,nfl.t.15,https://sports.yahoo.com/nfl/teams/miami/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/eDlltI2Ax54R...,https://s.yimg.com/iu/api/res/1.2/eDlltI2Ax54R...,False,False,False,0,Ollie,Ollie,Ollie Gordon II,Gordon II,41969,461.p.41969,O,RB,https://sports.yahoo.com/nfl/players/41969,2025


In [8]:
yahoo_id_maps = yahoo_players.set_index('name.full')['player_id'].to_dict()
master_ids.loc[master_ids['yahoo_id'].isna(), 'yahoo_id'] = master_ids.loc[master_ids['yahoo_id'].isna(), 'name'].map(yahoo_id_maps)


In [9]:
yahoo_id_maps['Ollie Gordon II']

41969

In [10]:
master_ids[master_ids['yahoo_id'].isna()]

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
29,17059,42753bb4-a416-4d40-bf81-f20d2181be2f,25984.0,00-0040198,157792.0,12495.0,58384.0,4711533.0,NaN,NaN,28873887.0,GordOl00,NaN,18450.0,NaN,1759.0,41969.0,0.0,25953.0,NaN,Ollie Gordon,ollie gordon,RB,MIA,2004-01-15,21.8,2025.0,6.0,3.0,179.0,NaN,74.0,225.0,Oklahoma State,2025
31,17061,14aedd06-41a6-41db-8c5e-05483f91570d,27285.0,00-0040238,158290.0,12544.0,58441.0,4911851.0,NaN,NaN,28877884.0,AlleLe01,NaN,18476.0,NaN,1761.0,42007.0,0.0,25986.0,NaN,LeQuint Allen,lequint allen,RB,JAC,2004-08-05,21.2,2025.0,7.0,20.0,236.0,NaN,72.0,201.0,Syracuse,2025
35,17065,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,Henry Parrish,henry parrish,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,70.0,185.0,Mississippi,2025
37,17067,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN,John Emery,john emery,RB,FA,NaN,NaN,2025.0,NaN,NaN,NaN,NaN,72.0,220.0,LSU,2025
40,17070,a4c21977-c9d1-40f8-abee-0862dfab9195,27016.0,00-0040735,157170.0,12519.0,58244.0,4685278.0,NaN,NaN,28874444.0,BurdLu00,NaN,18548.0,NaN,1770.0,41824.0,0.0,26044.0,NaN,Luther Burden,luther burden,WR,CHI,2003-12-12,21.8,2025.0,2.0,7.0,39.0,NaN,72.0,210.0,Missouri,2025
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
12139,968,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Sean Brewer,sean brewer,TE,CIN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12140,972,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Doug Brien,doug brien,PK,NOS,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12141,978,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Jeremy Brigham,jeremy brigham,TE,OAK,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025
12142,984,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,Vincent Brisby,vincent brisby,WR,FA,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,2025


IDs that are missing look to be a mismatch between JR/SR/II/III being listed as a suffix in one set but not the others. Need to standardize namaes, but first check if doing so would create any issues later

In [11]:
def check_cleaning_duplicates(original_dict, clean_function):
    """Check if cleaning will create duplicate keys"""
    
    # Map: cleaned_name -> list of original names
    cleaned_to_original = defaultdict(list)
    
    for original_name in original_dict.keys():
        cleaned = clean_function(original_name)
        cleaned_to_original[cleaned].append(original_name)
    
    # Find cleaned names that have multiple originals
    duplicates = {k: v for k, v in cleaned_to_original.items() if len(v) > 1}
    
    if duplicates:
        print(f"⚠️  Found {len(duplicates)} names that will become duplicates after cleaning:\n")
        for cleaned_name, original_names in duplicates.items():
            print(f"'{cleaned_name}' ←")
            for orig in original_names:
                print(f"    - '{orig}' (ID: {original_dict[orig]})")
            print()
        return duplicates
    else:
        print("✓ No duplicates will be created after cleaning")
        return {}

In [12]:

def clean_name(name):
    if pd.isna(name) or name is None:
        return name
    return re.sub(r'\s+(Jr\.?|Sr\.?|III|II|IV|V)(\s|$|,)', '', str(name), flags=re.IGNORECASE).strip()

In [13]:
check_cleaning_duplicates(yahoo_id_maps, clean_name)

⚠️  Found 1 names that will become duplicates after cleaning:

'Frank Gore' ←
    - 'Frank Gore' (ID: 7241)
    - 'Frank Gore Jr.' (ID: 41243)



{'Frank Gore': ['Frank Gore', 'Frank Gore Jr.']}

Gore and gore Jr are not missing any yahoo IDs so the code should not edit these IDs in any way, everything esle is covered

In [16]:
cln_yahoo_id_maps = {clean_name(key): value for key, value in yahoo_id_maps.items()}

In [17]:
master_ids.loc[master_ids['yahoo_id'].isna(), 'yahoo_id'] = master_ids.loc[master_ids['yahoo_id'].isna(), 'name'].map(cln_yahoo_id_maps)

In [36]:
# Any missing yahoo ID values are now because those players were not in the yahoo ID Set, can drop these rows
master_ids = master_ids.dropna(subset=['yahoo_id'])
master_ids['yahoo_id'] = master_ids['yahoo_id'].astype(int)
master_ids.info()

<class 'pandas.core.frame.DataFrame'>
Index: 3147 entries, 0 to 12138
Data columns (total 35 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   mfl_id            3147 non-null   int64  
 1   sportradar_id     3051 non-null   object 
 2   fantasypros_id    2485 non-null   float64
 3   gsis_id           2891 non-null   object 
 4   pff_id            2801 non-null   float64
 5   sleeper_id        2822 non-null   float64
 6   nfl_id            2773 non-null   float64
 7   espn_id           2944 non-null   float64
 8   yahoo_id          3147 non-null   int64  
 9   fleaflicker_id    297 non-null    float64
 10  cbs_id            3040 non-null   float64
 11  pfr_id            2785 non-null   object 
 12  cfbref_id         1362 non-null   object 
 13  rotowire_id       3108 non-null   float64
 14  rotoworld_id      2025 non-null   float64
 15  ktc_id            393 non-null    float64
 16  stats_id          3098 non-null   float64
 17 

In [37]:
yahoo_players['player_id'].nunique()

4263

In [38]:
master_ids['yahoo_id'].nunique()

3100

In [49]:
yahoo_players[yahoo_players['name.full'].str.startswith('Andrew')].head(50)

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
299,9,TE,nfl.p.24130,GB,Green Bay Packers,nfl.t.9,https://sports.yahoo.com/nfl/teams/green-bay/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/n1WorK4gEZDK...,https://s.yimg.com/iu/api/res/1.2/n1WorK4gEZDK...,False,False,False,0,Andrew,Andrew,Andrew Quarless,Quarless,24130,331.p.24130,O,TE,https://sports.yahoo.com/nfl/players/24130,2014
352,4,WR,nfl.p.24774,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/OMqXTfJd40ld...,https://s.yimg.com/iu/api/res/1.2/OMqXTfJd40ld...,False,False,False,0,Andrew,Andrew,Andrew Hawkins,Hawkins,24774,331.p.24774,O,WR,https://sports.yahoo.com/nfl/players/24774,2014
483,10,QB,nfl.p.25711,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/py29KAjMiTYt...,https://s.yimg.com/iu/api/res/1.2/py29KAjMiTYt...,False,False,False,0,Andrew,Andrew,Andrew Luck,Luck,25711,331.p.25711,O,QB,https://sports.yahoo.com/nfl/players/25711,2014
583,9,TE,nfl.p.26076,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/tG2QjQ7FE6UI...,https://s.yimg.com/iu/api/res/1.2/tG2QjQ7FE6UI...,False,False,False,0,Andrew,Andrew,Andrew Szczerba,Szczerba,26076,331.p.26076,O,TE,https://sports.yahoo.com/nfl/players/26076,2014
646,10,TE,nfl.p.26583,Min,Minnesota Vikings,nfl.t.16,https://sports.yahoo.com/nfl/teams/minnesota/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/f0hoO_Wz9sSq...,https://s.yimg.com/iu/api/res/1.2/f0hoO_Wz9sSq...,False,False,False,0,Andrew,Andrew,Andrew DePaola,DePaola,26583,331.p.26583,O,TE,https://sports.yahoo.com/nfl/players/26583,2014
1006,9,WR,nfl.p.28008,Det,Detroit Lions,nfl.t.8,https://sports.yahoo.com/nfl/teams/detroit/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/2TQwM3Oot.p2...,https://s.yimg.com/iu/api/res/1.2/2TQwM3Oot.p2...,False,False,False,0,Andrew,Andrew,Andrew Peacock,Peacock,28008,331.p.28008,O,WR,https://sports.yahoo.com/nfl/players/28008,2014
1125,11,K,nfl.p.28313,NYJ,New York Jets,nfl.t.20,https://sports.yahoo.com/nfl/teams/ny-jets/,['K'],small,https://s.yimg.com/iu/api/res/1.2/ilCNU_twT9fy...,https://s.yimg.com/iu/api/res/1.2/ilCNU_twT9fy...,False,False,False,0,Andrew,Andrew,Andrew Furney,Furney,28313,331.p.28313,K,K,https://sports.yahoo.com/nfl/players/28313,2014
1395,7,TE,nfl.p.24130,GB,Green Bay Packers,nfl.t.9,https://sports.yahoo.com/nfl/teams/green-bay/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/n1WorK4gEZDK...,https://s.yimg.com/iu/api/res/1.2/n1WorK4gEZDK...,False,False,False,0,Andrew,Andrew,Andrew Quarless,Quarless,24130,348.p.24130,O,TE,https://sports.yahoo.com/nfl/players/24130,2015
1429,11,WR,nfl.p.24774,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/OMqXTfJd40ld...,https://s.yimg.com/iu/api/res/1.2/OMqXTfJd40ld...,False,False,False,0,Andrew,Andrew,Andrew Hawkins,Hawkins,24774,348.p.24774,O,WR,https://sports.yahoo.com/nfl/players/24774,2015
1529,10,QB,nfl.p.25711,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/py29KAjMiTYt...,https://s.yimg.com/iu/api/res/1.2/py29KAjMiTYt...,False,False,False,0,Andrew,Andrew,Andrew Luck,Luck,25711,348.p.25711,O,QB,https://sports.yahoo.com/nfl/players/25711,2015


In [56]:
master_ids.head()

,mfl_id,sportradar_id,fantasypros_id,gsis_id,pff_id,sleeper_id,nfl_id,espn_id,yahoo_id,fleaflicker_id,cbs_id,pfr_id,cfbref_id,rotowire_id,rotoworld_id,ktc_id,stats_id,stats_global_id,fantasy_data_id,swish_id,name,merge_name,position,team,birthdate,age,draft_year,draft_round,draft_pick,draft_ovr,twitter_username,height,weight,college,db_season
0,17030,3c76cab3-3df2-43dd-acaa-57e055bd32d0,24755.0,00-0040676,133244.0,12522.0,58203.0,4688380.0,41786,NaN,3168422.0,WardCa00,NaN,16997.0,NaN,1730.0,41786.0,0.0,25323.0,NaN,Cam Ward,cam ward,QB,TEN,2002-05-25,23.4,2025.0,1.0,1.0,1.0,NaN,74.0,219.0,Miami (FL),2025
1,17031,270e09bc-8bf8-44b0-87ed-1fd014de4ab7,25968.0,00-0040668,131396.0,12524.0,58349.0,4432762.0,41930,NaN,26710004.0,SandSh00,NaN,18479.0,NaN,1731.0,41930.0,0.0,26079.0,NaN,Shedeur Sanders,shedeur sanders,QB,CLE,2002-02-07,23.7,2025.0,5.0,6.0,144.0,NaN,74.0,212.0,Colorado,2025
2,17032,53848cba-bcad-4cd0-bb35-0a1f6ed111d1,23160.0,00-0040691,146409.0,12508.0,58227.0,4689114.0,41810,NaN,26694019.0,DartJa00,NaN,18574.0,NaN,1732.0,41810.0,0.0,26082.0,NaN,Jaxson Dart,jaxson dart,QB,NYG,2003-05-13,22.4,2025.0,1.0,25.0,25.0,NaN,74.0,223.0,Ole Miss,2025
3,17033,b0eb5b67-4ef2-462d-bfe7-e73bcce1f6b0,22884.0,00-0040673,146590.0,12510.0,58297.0,4432734.0,41882,NaN,26694943.0,MilrJa00,NaN,18477.0,NaN,1733.0,41882.0,0.0,25882.0,NaN,Jalen Milroe,jalen milroe,QB,SEA,2002-12-13,22.8,2025.0,3.0,28.0,92.0,NaN,74.0,216.0,Alabama,2025
4,17034,d89f0384-21a6-44e4-8aa6-324cb2103bf6,23680.0,00-0040234,127253.0,12500.0,58436.0,4889929.0,42037,NaN,26746721.0,EwerQu00,NaN,18505.0,NaN,1734.0,42037.0,0.0,26048.0,NaN,Quinn Ewers,quinn ewers,QB,MIA,2003-05-13,22.4,2025.0,7.0,15.0,231.0,NaN,74.0,209.0,Texas,2025


In [59]:
dump_players = master_ids[~master_ids['yahoo_id'].isin(yahoo_players['player_id'])]

In [61]:
master_ids = master_ids.drop(dump_players, axis = 1)

In [63]:
master_ids.head()

""
0
1
2
3
4


In [62]:
master_ids[~master_ids['yahoo_id'].isin(yahoo_players['player_id'])]

KeyError: 'yahoo_id'

In [33]:
yahoo_players[yahoo_players['player_id'].isin(master_ids['yahoo_id'])]

,bye_weeks.week,display_position,editorial_player_key,editorial_team_abbr,editorial_team_full_name,editorial_team_key,editorial_team_url,eligible_positions,headshot.size,headshot.url,image_url,is_keeper.status,is_keeper.cost,is_keeper.kept,is_undroppable,name.ascii_first,name.first,name.full,name.last,player_id,player_key,position_type,primary_position,url,season
0,10,K,nfl.p.3727,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['K'],small,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,https://s.yimg.com/iu/api/res/1.2/OpHvpCHjl_PQ...,False,False,False,0,Adam,Adam,Adam Vinatieri,Vinatieri,3727,331.p.3727,K,K,https://sports.yahoo.com/nfl/players/3727,2014
1,9,TE,nfl.p.3950,Atl,Atlanta Falcons,nfl.t.1,https://sports.yahoo.com/nfl/teams/atlanta/,['TE'],small,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,https://s.yimg.com/iu/api/res/1.2/1_ZQRTBuF_fZ...,False,False,False,0,Tony,Tony,Tony Gonzalez,Gonzalez,3950,331.p.3950,O,TE,https://sports.yahoo.com/nfl/players/3950,2014
2,4,QB,nfl.p.4256,Den,Denver Broncos,nfl.t.7,https://sports.yahoo.com/nfl/teams/denver/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,https://s.yimg.com/iu/api/res/1.2/UCWtIm9kEbbH...,False,False,False,0,Peyton,Peyton,Peyton Manning,Manning,4256,331.p.4256,O,QB,https://sports.yahoo.com/nfl/players/4256,2014
3,4,K,nfl.p.4269,Ari,Arizona Cardinals,nfl.t.22,https://sports.yahoo.com/nfl/teams/arizona/,['K'],small,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,https://s.yimg.com/iu/api/res/1.2/z4ki.EZ6BaZp...,False,False,False,0,Phil,Phil,Phil Dawson,Dawson,4269,331.p.4269,K,K,https://sports.yahoo.com/nfl/players/4269,2014
4,10,QB,nfl.p.4416,Ind,Indianapolis Colts,nfl.t.11,https://sports.yahoo.com/nfl/teams/indianapolis/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,https://s.yimg.com/iu/api/res/1.2/YdWdJ6P7MDCj...,False,False,False,0,Matt,Matt,Matt Hasselbeck,Hasselbeck,4416,331.p.4416,O,QB,https://sports.yahoo.com/nfl/players/4416,2014
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14044,11,QB,nfl.p.42537,NO,New Orleans Saints,nfl.t.18,https://sports.yahoo.com/nfl/teams/new-orleans/,['QB'],small,https://s.yimg.com/iu/api/res/1.2/SclDLq4JOthk...,https://s.yimg.com/iu/api/res/1.2/SclDLq4JOthk...,False,False,False,0,Hunter,Hunter,Hunter Dekkers,Dekkers,42537,461.p.42537,O,QB,https://sports.yahoo.com/nfl/players/42537,2025
14047,5,WR,nfl.p.42545,GB,Green Bay Packers,nfl.t.9,https://sports.yahoo.com/nfl/teams/green-bay/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/duAZD8ehn2.t...,https://s.yimg.com/iu/api/res/1.2/duAZD8ehn2.t...,False,False,False,0,Sam,Sam,Sam Brown Jr.,Brown Jr.,42545,461.p.42545,O,WR,https://sports.yahoo.com/nfl/players/42545,2025
14052,9,RB,nfl.p.42562,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['RB'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Toa,Toa,Toa Taua,Taua,42562,461.p.42562,O,RB,https://sports.yahoo.com/nfl/players/42562,2025
14061,9,WR,nfl.p.42598,Cle,Cleveland Browns,nfl.t.5,https://sports.yahoo.com/nfl/teams/cleveland/,['WR'],small,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,https://s.yimg.com/iu/api/res/1.2/TcM85WhJ.fAO...,False,False,False,0,Isaiah,Isaiah,Isaiah Bond,Bond,42598,461.p.42598,O,WR,https://sports.yahoo.com/nfl/players/42598,2025
